### Programming Assignment 2

#### Question 1
How many alignments does the naive exact matching algorithm try when matching the string GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG (derived from human Alu sequences) to the excerpt of human chromosome 1?  (Don't consider reverse complements.)

In [1]:
def naive_with_counts(p, t):
    occurrences = []
    num_alignments = 0
    num_character_comparisons = 0
    for i in range(len(t) - len(p) + 1):
        match = True
        num_alignments += 1
        for j in range(len(p)):
            num_character_comparisons += 1
            if t[i + j] != p[j]:
                match = False
                break
        if match:
            occurrences.append(i)
    
    return occurrences, num_alignments, num_character_comparisons

In [2]:
def readGenome(filename):
    genome = ''
    with open(filename,'r') as f:
        for line in f:
            if line[0] != '>':
                genome += line.rstrip()
    return genome

In [3]:
genome = readGenome('chr1.GRCh38.excerpt.fasta')
genome[:100]

'TTGAATGCTGAAATCAGCAGGTAATATATGATAATAGAGAAAGCTATCCCGAAGGTGCATAGGTCAACAATACTTGAGCCTAACTCAGTAGATCCTAAAA'

In [4]:
pattern = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
_, num_allignments, _ = naive_with_counts(pattern,genome)
num_allignments

799954

#### Question 2
How many character comparisons does the naive exact matching algorithm try when matching the string GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG (derived from human Alu sequences) to the excerpt of human chromosome 1?  (Don't consider reverse complements.)

In [5]:
pattern = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
_ , _ , num_character_comparisons = naive_with_counts(pattern, genome)
num_character_comparisons

984143

#### Question 3
How many alignments does Boyer-Moore try when matching the string GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG (derived from human Alu sequences) to the excerpt of human chromosome 1?  (Don't consider reverse complements.)

In [6]:
from bm_preproc import BoyerMoore

In [7]:
def boyer_moore_with_counts(p, p_bm, t):
    i = 0
    occurrences = []
    num_alignments = 0
    num_character_comparisons = 0
    while i < len(t) - len(p) + 1:
        shift = 1
        num_alignments += 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):
            num_character_comparisons += 1
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences, num_alignments, num_character_comparisons

In [10]:
pattern = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
p_bm = BoyerMoore(pattern, alphabet='ACGT')
_, num_alignments, _ = boyer_moore_with_counts(pattern, p_bm, genome)
num_alignments

127974

#### Question 4
Index-assisted approximate matching. In practicals, we built a Python class called Index

implementing an ordered-list version of the k-mer index.

We also implemented the pigeonhole principle using Boyer-Moore as our exact matching algorithm.

Implement the pigeonhole principle using Index to find exact matches for the partitions. Assume P always has length 24, and that we are looking for approximate matches with up to 2 mismatches (substitutions). We will use an 8-mer index.

Download the Python module for building a k-mer index. 

https://d28rh4a8wq0iu5.cloudfront.net/ads1/code/kmer_index.py

Write a function that, given a length-24 pattern P and given an Index object built on 8-mers, finds all approximate occurrences of P within T with up to 2 mismatches. Insertions and deletions are not allowed. Don't consider any reverse complements.

How many times does the string GGCGCGGTGGCTCACGCCTGTAAT, which is derived from a human Alu sequence, occur with up to 2 substitutions in the excerpt of human chromosome 1?  (Don't consider reverse complements here.)

Hint 1: Multiple index hits might direct you to the same match multiple times, but be careful not to count a match more than once.

Hint 2: You can check your work by comparing the output of your new function to that of the naive_2mm function implemented in the previous module.

In [11]:
from kmer_index import Index

def queryIndex(pattern, text, index : Index):
    k = index.k
    offsets = []
    for i in index.query(pattern) :
        if pattern[k:] == text[i + k : i + len(pattern)]:
            offsets.append(i)

    return offsets

In [73]:
def index_assisted_approximate_matching(pattern, genome, max_err,k):
    segment_len = int(round(len(pattern) / ( max_err + 1 )))
    all_matches = set()
    index = Index(genome,k)
    for i in range(max_err + 1):
        start = i * segment_len
        end = min ((i + 1) * segment_len, len(pattern))
        matches = queryIndex(pattern[start:end],genome,index)
        for m in matches:
            if m < start or m - start + len(pattern) > len(genome):
                continue

            mismatches = 0
            for j in range(0,start):
                if not pattern[j] == genome[m - start + j]:
                    mismatches += 1
                    if mismatches > max_err:
                        break
            
            for j in range(end,len(pattern)):
                if not pattern[j] == genome[m - start + j]:
                    mismatches += 1
                    if mismatches > max_err:
                        break
            
            if mismatches <= max_err:
                all_matches.add(m - start)
                
    return list(all_matches)

In [85]:
pattern = 'GGCGCGGTGGCTCACGCCTGTAAT'
tot_matches = index_assisted_approximate_matching(pattern,genome,2,8)
len(tot_matches)

19

#### Question 6
Let's examine whether there is a benefit to using an index built using subsequences of T rather than substrings, as we discussed in the "Variations on k-mer indexes" video.  We'll consider subsequences involving every N characters.  For example, if we split ATATAT into two substring partitions, we would get partitions ATA (the first half) and TAT (second half).  But if we split ATATAT into two  subsequences  by taking every other character, we would get AAA (first, third and fifth characters) and TTT (second, fourth and sixth).

Another way to visualize this is using numbers to show how each character of P is allocated to a partition.  Splitting a length-6 pattern into two substrings could be represented as 111222, and splitting into two subsequences of every other character could be represented as 121212

The following class SubseqIndex is a more general implementation of Index that additionally handles subsequences. It only considers subsequences that take every Nth character:

Write a function that, given a length-24 pattern P and given a SubseqIndex object built with k = 8 and ival = 3, finds all approximate occurrences of P within T with up to 2 mismatches.

When using this function, how many total index hits are there when searching for GGCGCGGTGGCTCACGCCTGTAAT with up to 2 substitutions in the excerpt of human chromosome 1?  (Again, don't consider reverse complements.)

In [79]:
import bisect
   
class SubseqIndex(object):
    """ Holds a subsequence index for a text T """
    
    def __init__(self, t, k, ival):
        """ Create index from all subsequences consisting of k characters
            spaced ival positions apart.  E.g., SubseqIndex("ATAT", 2, 2)
            extracts ("AA", 0) and ("TT", 1). """
        self.k = k  # num characters per subsequence extracted
        self.ival = ival  # space between them; 1=adjacent, 2=every other, etc
        self.index = []
        self.span = 1 + ival * (k - 1)
        for i in range(len(t) - self.span + 1):  # for each subseq
            self.index.append((t[i:i+self.span:ival], i))  # add (subseq, offset)
        self.index.sort()  # alphabetize by subseq
    
    def query(self, p):
        """ Return index hits for first subseq of p """
        subseq = p[:self.span:self.ival]  # query with first subseq
        i = bisect.bisect_left(self.index, (subseq, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != subseq:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits


In [83]:
ind = SubseqIndex('ATATAT', 2, 2)
print(ind.index)


[('AA', 0), ('AA', 2), ('TT', 1), ('TT', 3)]


Incomplete...

In [88]:
def subs_index_approximate_matching(pattern, genome, subs_ind: SubseqIndex, max_err):
    segment_len = int(round(len(pattern) / ( max_err + 1 )))
    all_matches = set()
    for i in range(max_err + 1):
        start = i * segment_len
        end = min ((i + 1) * segment_len, len(pattern))
        matches = subs_ind.query(pattern[start:end])
        print(matches)
        for m in matches:
            if m < start or m - start + len(pattern) > len(genome):
                continue

            mismatches = 0
            for j in range(0,start):
                if not pattern[j] == genome[m - start + j]:
                    mismatches += 1
                    if mismatches > max_err:
                        break
            
            for j in range(end,len(pattern)):
                if not pattern[j] == genome[m - start + j]:
                    mismatches += 1
                    if mismatches > max_err:
                        break
            
            if mismatches <= max_err:
                all_matches.add(m - start)
                
    return list(all_matches)

In [89]:

t = 'to-morrow and to-morrow and to-morrow creeps in this petty pace'
p = 'to-morrow and to-morrow '
subseq_ind = SubseqIndex(t, 8, 3)
subs_index_approximate_matching(p, t, subseq_ind, 2)

[]
[]
[]


[]